In [1]:
"""
pacfile.py -- Defines a PACFile class to handle reading and writing audio
data to an audio file holding data compressed using an MDCT-based perceptual audio
coding algorithm.  The MDCT lines of each audio channel are grouped into bands,
each sharing a single scaleFactor and bit allocation that are used to block-
floating point quantize those lines.  This class is a subclass of AudioFile.

-----------------------------------------------------------------------
© 2009 Marina Bosi & Richard E. Goldberg -- All rights reserved
-----------------------------------------------------------------------

See the documentation of the AudioFile class for general use of the AudioFile
class.

Notes on reading and decoding PAC files:

    The OpenFileForReading() function returns a CodedParams object containing:

        nChannels = the number of audio channels
        sampleRate = the sample rate of the audio samples
        numSamples = the total number of samples in the file for each channel
        nMDCTLines = half the MDCT block size (block switching not supported)
        nSamplesPerBlock = MDCTLines (but a name that PCM files look for)
        nScaleBits = the number of bits storing scale factors
        nMantSizeBits = the number of bits storing mantissa bit allocations
        sfBands = a ScaleFactorBands object
        overlapAndAdd = decoded data from the prior block (initially all zeros)

    The returned ScaleFactorBands object, sfBands, contains an allocation of
    the MDCT lines into groups that share a single scale factor and mantissa bit
    allocation.  sfBands has the following attributes available:

        nBands = the total number of scale factor bands
        nLines[iBand] = the number of MDCT lines in scale factor band iBand
        lowerLine[iBand] = the first MDCT line in scale factor band iBand
        upperLine[iBand] = the last MDCT line in scale factor band iBand


Notes on encoding and writing PAC files:

    When writing to a PACFile the CodingParams object passed to OpenForWriting()
    should have the following attributes set:

        nChannels = the number of audio channels
        sampleRate = the sample rate of the audio samples
        numSamples = the total number of samples in the file for each channel
        nMDCTLines = half the MDCT block size (format does not support block switching)
        nSamplesPerBlock = MDCTLines (but a name that PCM files look for)
        nScaleBits = the number of bits storing scale factors
        nMantSizeBits = the number of bits storing mantissa bit allocations
        targetBitsPerSample = the target encoding bit rate in units of bits per sample

    The first three attributes (nChannels, sampleRate, and numSamples) are
    typically added by the original data source (e.g. a PCMFile object) but
    numSamples may need to be extended to account for the MDCT coding delay of
    nMDCTLines and any zero-padding done in the final data block

    OpenForWriting() will add the following attributes to be used during the encoding
    process carried out in WriteDataBlock():

        sfBands = a ScaleFactorBands object
        priorBlock = the prior block of audio data (initially all zeros)

    The passed ScaleFactorBands object, sfBands, contains an allocation of
    the MDCT lines into groups that share a single scale factor and mantissa bit
    allocation.  sfBands has the following attributes available:

        nBands = the total number of scale factor bands
        nLines[iBand] = the number of MDCT lines in scale factor band iBand
        lowerLine[iBand] = the first MDCT line in scale factor band iBand
        upperLine[iBand] = the last MDCT line in scale factor band iBand

Description of the PAC File Format:

    Header:

        tag                 4 byte file tag equal to "PAC "
        sampleRate          little-endian unsigned long ("<L" format in struct)
        nChannels           little-endian unsigned short("<H" format in struct)
        numSamples          little-endian unsigned long ("<L" format in struct)
        nMDCTLines          little-endian unsigned long ("<L" format in struct)
        nScaleBits          little-endian unsigned short("<H" format in struct)
        nMantSizeBits       little-endian unsigned short("<H" format in struct)
        nSFBands            little-endian unsigned long ("<L" format in struct)
        for iBand in range(nSFBands):
            nLines[iBand]   little-endian unsigned short("<H" format in struct)

    Each Data Block:  (reads data blocks until end of file hit)

        for iCh in range(nChannels):
            nBytes          little-endian unsigned long ("<L" format in struct)
            as bits packed into an array of nBytes bytes:
                overallScale[iCh]                       nScaleBits bits
                for iBand in range(nSFBands):
                    scaleFactor[iCh][iBand]             nScaleBits bits
                    bitAlloc[iCh][iBand]                nMantSizeBits bits
                    if bitAlloc[iCh][iBand]:
                        for m in nLines[iBand]:
                            mantissa[iCh][iBand][m]     bitAlloc[iCh][iBand]+1 bits
                <extra custom data bits as long as space is included in nBytes>

"""

from audiofile import * # base class
from bitpack import *  # class for packing data into an array of bytes where each item's number of bits is specified
import codec    # module where the actual PAC coding functions reside(this module only specifies the PAC file format)
from psychoac import ScaleFactorBands, AssignMDCTLinesFromFreqLimits  # defines the grouping of MDCT lines into scale factor bands

import numpy as np  # to allow conversion of data blocks to numpy's array object
MAX16BITS = 32767


class PACFile(AudioFile):
    """
    Handlers for a perceptually coded audio file I am encoding/decoding
    """

    # a file tag to recognize PAC coded files
    tag=b'PAC '

    def ReadFileHeader(self):
        """
        Reads the PAC file header from a just-opened PAC file and uses it to set
        object attributes.  File pointer ends at start of data portion.
        """
        # check file header tag to make sure it is the right kind of file
        tag=self.fp.read(4)
        if tag!=self.tag: raise RuntimeError("Tried to read a non-PAC file into a PACFile object")
        # use struct.unpack() to load up all the header data
        (sampleRate, nChannels, numSamples, nMDCTLines, nScaleBits, nMantSizeBits) \
                 = unpack('<LHLLHH',self.fp.read(calcsize('<LHLLHH')))
        nBands = unpack('<L',self.fp.read(calcsize('<L')))[0]
        nLines=  unpack('<'+str(nBands)+'H',self.fp.read(calcsize('<'+str(nBands)+'H')))
        sfBands=ScaleFactorBands(nLines)
        # load up a CodingParams object with the header data
        myParams=CodingParams()
        myParams.sampleRate = sampleRate
        myParams.nChannels = nChannels
        myParams.numSamples = numSamples
        myParams.nMDCTLines = myParams.nSamplesPerBlock = nMDCTLines
        myParams.nScaleBits = nScaleBits
        myParams.nMantSizeBits = nMantSizeBits
        # add in scale factor band information
        myParams.sfBands =sfBands
        # start w/o all zeroes as data from prior block to overlap-and-add for output
        overlapAndAdd = []
        for iCh in range(nChannels): overlapAndAdd.append( np.zeros(nMDCTLines, dtype=np.float64) )
        myParams.overlapAndAdd=overlapAndAdd
        return myParams


    def ReadDataBlock(self, codingParams):
        """
        Reads a block of coded data from a PACFile object that has already
        executed OpenForReading() and returns those samples as reconstituted
        signed-fraction data
        """
        # loop over channels (whose coded data are stored separately) and read in each data block
        data=[]
        for iCh in range(codingParams.nChannels):
            data.append(np.array([],dtype=np.float64))  # add location for this channel's data
            # read in string containing the number of bytes of data for this channel (but check if at end of file!)
            s=self.fp.read(calcsize("<L"))  # will be empty if at end of file
            if not s:
                # hit last block, see if final overlap and add needs returning, else return nothing
                if codingParams.overlapAndAdd:
                    overlapAndAdd=codingParams.overlapAndAdd
                    codingParams.overlapAndAdd=0  # setting it to zero so next pass will just return
                    return overlapAndAdd
                else:
                    return
            # not at end of file, get nBytes from the string we just read
            nBytes = unpack("<L",s)[0] # read it as a little-endian unsigned long
            # read the nBytes of data into a PackedBits object to unpack
            pb = PackedBits()
            pb.SetPackedData( self.fp.read(nBytes) ) # PackedBits function SetPackedData() converts strings to internally-held array of bytes
            if pb.nBytes < nBytes:  raise "Only read a partial block of coded PACFile data"

            # extract the data from the PackedBits object
            overallScaleFactor = pb.ReadBits(codingParams.nScaleBits)  # overall scale factor
            scaleFactor=[]
            bitAlloc=[]
            mantissa=np.zeros(codingParams.nMDCTLines,np.int32)  # start w/ all mantissas zero
            for iBand in range(codingParams.sfBands.nBands): # loop over each scale factor band to pack its data
                ba = pb.ReadBits(codingParams.nMantSizeBits)
                if ba: ba+=1  # no bit allocation of 1 so ba of 2 and up stored as one less
                bitAlloc.append(ba)  # bit allocation for this band
                scaleFactor.append(pb.ReadBits(codingParams.nScaleBits))  # scale factor for this band
                if bitAlloc[iBand]:
                    # if bits allocated, extract those mantissas and put in correct location in matnissa array
                    m=np.empty(codingParams.sfBands.nLines[iBand],np.int32)
                    for j in range(codingParams.sfBands.nLines[iBand]):
                        m[j]=pb.ReadBits(bitAlloc[iBand])     # mantissas for this band (if bit allocation non-zero) and bit alloc <>1 so encoded as 1 lower than actual allocation
                    mantissa[codingParams.sfBands.lowerLine[iBand]:(codingParams.sfBands.upperLine[iBand]+1)] = m
            # done unpacking data (end loop over scale factor bands)

            # CUSTOM DATA:
            # < now can unpack any custom data passed in the nBytes of data >

            # (DECODE HERE) decode the unpacked data for this channel, overlap-and-add first half, and append it to the data array (saving other half for next overlap-and-add)
            decodedData = self.Decode(scaleFactor,bitAlloc,mantissa, overallScaleFactor,codingParams)
            data[iCh] = np.concatenate( (data[iCh],np.add(codingParams.overlapAndAdd[iCh],decodedData[:codingParams.nMDCTLines]) ) )  # data[iCh] is overlap-and-added data
            codingParams.overlapAndAdd[iCh] = decodedData[codingParams.nMDCTLines:]  # save other half for next pass

        # end loop over channels, return signed-fraction samples for this block
        return data


    def WriteFileHeader(self,codingParams):
        """
        Writes the PAC file header for a just-opened PAC file and uses codingParams
        attributes for the header data.  File pointer ends at start of data portion.
        """
        # write a header tag
        self.fp.write(self.tag)
        # make sure that the number of samples in the file is a multiple of the
        # number of MDCT half-blocksize, otherwise zero pad as needed
        if not codingParams.numSamples%codingParams.nMDCTLines:
            codingParams.numSamples += (codingParams.nMDCTLines
                        - codingParams.numSamples%codingParams.nMDCTLines) # zero padding for partial final PCM block
        # also add in the delay block for the second pass w/ the last half-block
        codingParams.numSamples+= codingParams.nMDCTLines  # due to the delay in processing the first samples on both sides of the MDCT block
        # write the coded file attributes
        self.fp.write(pack('<LHLLHH',
            codingParams.sampleRate, codingParams.nChannels,
            codingParams.numSamples, codingParams.nMDCTLines,
            codingParams.nScaleBits, codingParams.nMantSizeBits  ))
        # create a ScaleFactorBand object to be used by the encoding process and write its info to header
        sfBands=ScaleFactorBands( AssignMDCTLinesFromFreqLimits(codingParams.nMDCTLines,
                                                                codingParams.sampleRate)
                                )
        codingParams.sfBands=sfBands
        self.fp.write(pack('<L',sfBands.nBands))
        self.fp.write(pack('<'+str(sfBands.nBands)+'H',*(sfBands.nLines.tolist()) ))
        # start w/o all zeroes as prior block of unencoded data for other half of MDCT block
        priorBlock = []
        for iCh in range(codingParams.nChannels):
            priorBlock.append(np.zeros(codingParams.nMDCTLines,dtype=np.float64) )
        codingParams.priorBlock = priorBlock
        return


    def WriteDataBlock(self,data, codingParams):
        """
        Writes a block of signed-fraction data to a PACFile object that has
        already executed OpenForWriting()"""

        # combine this block of multi-channel data w/ the prior block's to prepare for MDCTs twice as long
        fullBlockData=[]
        for iCh in range(codingParams.nChannels):
            fullBlockData.append( np.concatenate( ( codingParams.priorBlock[iCh], data[iCh]) ) )
        codingParams.priorBlock = data  # current pass's data is next pass's prior block data

        # (ENCODE HERE) Encode the full block of multi=channel data
        (scaleFactor,bitAlloc,mantissa, overallScaleFactor, codingParams.bitReservoir) = self.Encode(fullBlockData,codingParams)  # returns a tuple with all the block-specific info not in the file header
        
        # for each channel, write the data to the output file
        for iCh in range(codingParams.nChannels):

            # determine the size of this channel's data block and write it to the output file
            nBytes =codingParams.nScaleBits  # bits for overall scale factor
            for iBand in range(codingParams.sfBands.nBands): # loop over each scale factor band to get its bits
                nBytes += codingParams.nMantSizeBits+codingParams.nScaleBits    # mantissa bit allocation and scale factor for that sf band
                if bitAlloc[iCh][iBand]:
                    # if non-zero bit allocation for this band, add in bits for scale factor and each mantissa (0 bits means zero)
                    nBytes += bitAlloc[iCh][iBand]*codingParams.sfBands.nLines[iBand]  # no bit alloc = 1 so actuall alloc is one higher
            # end computing bits needed for this channel's data

            # CUSTOM DATA:
            # < now can add space for custom data, if desired>

            # now convert the bits to bytes (w/ extra one if spillover beyond byte boundary)
            if nBytes%BYTESIZE==0:  nBytes //= BYTESIZE
            else: nBytes = nBytes//BYTESIZE + 1
            self.fp.write(pack("<L",int(nBytes))) # stores size as a little-endian unsigned long

            # create a PackedBits object to hold the nBytes of data for this channel/block of coded data
            pb = PackedBits()
            pb.Size(nBytes)

            # now pack the nBytes of data into the PackedBits object
            pb.WriteBits(overallScaleFactor[iCh],codingParams.nScaleBits)  # overall scale factor
            iMant=0  # index offset in mantissa array (because mantissas w/ zero bits are omitted)
            for iBand in range(codingParams.sfBands.nBands): # loop over each scale factor band to pack its data
                ba = bitAlloc[iCh][iBand]
                if ba: ba-=1  # if non-zero, store as one less (since no bit allocation of 1 bits/mantissa)
                pb.WriteBits(ba,codingParams.nMantSizeBits)  # bit allocation for this band (written as one less if non-zero)
                pb.WriteBits(scaleFactor[iCh][iBand],codingParams.nScaleBits)  # scale factor for this band (if bit allocation non-zero)
                if bitAlloc[iCh][iBand]:
                    for j in range(codingParams.sfBands.nLines[iBand]):
                        pb.WriteBits(mantissa[iCh][iMant+j],bitAlloc[iCh][iBand])     # mantissas for this band (if bit allocation non-zero) and bit alloc <>1 so is 1 higher than the number
                    iMant += codingParams.sfBands.nLines[iBand]  # add to mantissa offset if we passed mantissas for this band
            # done packing (end loop over scale factor bands)

            # CUSTOM DATA:
            # < now can add in custom data if space allocated in nBytes above>

            # finally, write the data in this channel's PackedBits object to the output file
            self.fp.write(pb.GetPackedData())
        # end loop over channels, done writing coded data for all channels
        return codingParams.bitReservoir

    def Close(self,codingParams):
        """
        Flushes the last data block through the encoding process (if encoding)
        and closes the audio file
        """
        # determine if encoding or encoding and, if encoding, do last block
        if self.fp.mode == "wb":  # we are writing to the PACFile, must be encode
            # we are writing the coded file -- pass a block of zeros to move last data block to other side of MDCT block
            data = [ np.zeros(codingParams.nMDCTLines,dtype=np.float),
                     np.zeros(codingParams.nMDCTLines,dtype=np.float) ]
            self.WriteDataBlock(data, codingParams)
        self.fp.close()


    def Encode(self,data,codingParams):
        """
        Encodes multichannel audio data and returns a tuple containing
        the scale factors, mantissa bit allocations, quantized mantissas,
        and the overall scale factor for each channel.
        """
        #Passes encoding logic to the Encode function defined in the codec module
        return codec.Encode(data,codingParams)

    def Decode(self,scaleFactor,bitAlloc,mantissa, overallScaleFactor,codingParams):
        """
        Decodes a single audio channel of data based on the values of its scale factors,
        bit allocations, quantized mantissas, and overall scale factor.
        """
        #Passes decoding logic to the Decode function defined in the codec module
        return codec.Decode(scaleFactor,bitAlloc,mantissa, overallScaleFactor,codingParams)


#-----------------------------------------------------------------------------

# Testing the full PAC coder (needs a file called "input.wav" in the code directory)
if __name__=="__main__":
    
    filenames = ["castanet"]#["castanet", "glockenspiel", "harpsichord", "spfe", "spmg"]  

    for this_file in filenames:
 
        print( "\nTesting the PAC coder (input.wav -> coded.pac -> output.wav):")
        import time
        from pcmfile import * # to get access to WAV file handling
        elapsed = time.time()

        for Direction in ("Encode", "Decode"):
    #    for Direction in ("Decode"):

            # create the audio file objects
            if Direction == "Encode":
                print( "\n\tEncoding input PCM file...",)
                inFile= PCMFile(this_file + ".wav")
                outFile = PACFile("coded.pac")
            else: # "Decode"
                print( "\n\tDecoding coded PAC file...",)
                inFile = PACFile("coded.pac")
                outFile= PCMFile(this_file + "_out.wav")
            # only difference is file names and type of AudioFile object

            # open input file
            codingParams=inFile.OpenForReading()  # (includes reading header)

            # pass parameters to the output file
            if Direction == "Encode":
                # set additional parameters that are needed for PAC file
                # (beyond those set by the PCM file on open)
                codingParams.nMDCTLines = 512 #1024
                codingParams.nScaleBits = 3
                codingParams.nMantSizeBits = 5
                codingParams.targetBitsPerSample = 2.9 #(2.9 -> FOR 128 kb/s, 4.35 -> FOR 192 kb/s) 
                
                # initialize bit reservoir
                codingParams.bitReservoir = 0; 
                
                print("nMDCTLines = ", codingParams.nMDCTLines)
                print("nScaleBits = ", codingParams.nScaleBits)
                print("nMantSizeBits = ", codingParams.nMantSizeBits)
                print("targetBitsSample = ", codingParams.targetBitsPerSample)
                
                # tell the PCM file how large the block size is
                codingParams.nSamplesPerBlock = codingParams.nMDCTLines
                print(codingParams.nSamplesPerBlock)
            else: # "Decode"
                # set PCM parameters (the rest is same as set by PAC file on open)
                codingParams.bitsPerSample = 16
                codingParams.bitReservoir = 0
            # only difference is in setting up the output file parameters


            # open the output file
            outFile.OpenForWriting(codingParams) # (includes writing header)

            # Read the input file and pass its data to the output file to be written
            while True:
                data=inFile.ReadDataBlock(codingParams)
                if not data: break  # we hit the end of the input file
                codingParams.bitReservoir = outFile.WriteDataBlock(data,codingParams)
                print("codingParams.bitReservoir", codingParams.bitReservoir)
                print(" --------- ")
                print( ".",end="")  # just to signal how far we've gotten to user
            # end loop over reading/writing the blocks

            # close the files
            inFile.Close(codingParams)
            outFile.Close(codingParams)
        # end of loop over Encode/Decode

        elapsed = time.time()-elapsed
        print( "\nDone with Encode/Decode test\n")
        print( elapsed ," seconds elapsed")




Testing the PAC coder (input.wav -> coded.pac -> output.wav):

	Encoding input PCM file...
nMDCTLines =  512
nScaleBits =  3
nMantSizeBits =  5
targetBitsSample =  2.9
512
init bitBudget 1281.8
init bitBudget 2563.6
codingParams.bitReservoir 2563.6
 --------- 
.init bitBudget 3845.3999999999996
init bitBudget 5127.2
codingParams.bitReservoir 5127.2
 --------- 
.init bitBudget 6409.0
init bitBudget 7690.8
codingParams.bitReservoir 7690.8
 --------- 
.init bitBudget 8972.6
init bitBudget 10254.4
codingParams.bitReservoir 10254.4
 --------- 
.init bitBudget 11536.199999999999
init bitBudget 12817.999999999998
codingParams.bitReservoir 12817.999999999998
 --------- 
.init bitBudget 14099.799999999997
init bitBudget 15381.599999999997
codingParams.bitReservoir 15381.599999999997
 --------- 
.init bitBudget 16663.399999999998
init bitBudget 17945.199999999997
codingParams.bitReservoir 17945.199999999997
 --------- 
.init bitBudget 19226.999999999996
init bitBudget 20508.799999999996
codingP

codingParams.bitReservoir 144482.3999999999
 --------- 
.init bitBudget 145764.1999999999
init bitBudget 146438.99999999988
codingParams.bitReservoir 145705.99999999988
 --------- 
.init bitBudget 146987.79999999987
init bitBudget 147664.59999999986
codingParams.bitReservoir 146921.59999999986
 --------- 
.init bitBudget 148203.39999999985
init bitBudget 148759.19999999984
codingParams.bitReservoir 148118.19999999984
 --------- 
.init bitBudget 149399.99999999983
init bitBudget 149977.7999999998
codingParams.bitReservoir 149235.7999999998
 --------- 
.init bitBudget 150517.5999999998
bits used 1073
init bitBudget 150732.3999999998
bits used 1058
codingParams.bitReservoir 149674.3999999998
 --------- 
.init bitBudget 150956.19999999978
bits used 1156
init bitBudget 151084.99999999977
bits used 1123
codingParams.bitReservoir 149966.99999999977
 --------- 
.init bitBudget 151248.79999999976
init bitBudget 151717.59999999974
codingParams.bitReservoir 150962.59999999974
 --------- 
.init bi

.init bitBudget 205156.19999999838
init bitBudget 205657.99999999837
codingParams.bitReservoir 204873.99999999837
 --------- 
.init bitBudget 206155.79999999836
init bitBudget 206632.59999999835
codingParams.bitReservoir 205841.59999999835
 --------- 
.init bitBudget 207123.39999999834
init bitBudget 207588.19999999832
codingParams.bitReservoir 206806.19999999832
 --------- 
.init bitBudget 208087.9999999983
init bitBudget 208461.7999999983
codingParams.bitReservoir 207551.7999999983
 --------- 
.init bitBudget 208833.5999999983
init bitBudget 209293.39999999828
bits used 1022
codingParams.bitReservoir 208271.39999999828
 --------- 
.init bitBudget 209553.19999999827
init bitBudget 210141.99999999825
codingParams.bitReservoir 209422.99999999825
 --------- 
.init bitBudget 210704.79999999824
init bitBudget 211036.59999999823
codingParams.bitReservoir 210289.59999999823
 --------- 
.init bitBudget 211571.39999999822
init bitBudget 211922.1999999982
codingParams.bitReservoir 211023.199999

bits used 1249
codingParams.bitReservoir 267951.59999999683
 --------- 
.init bitBudget 269233.3999999968
init bitBudget 269551.1999999968
codingParams.bitReservoir 268600.1999999968
 --------- 
.init bitBudget 269881.9999999968
init bitBudget 270327.7999999968
codingParams.bitReservoir 269416.7999999968
 --------- 
.init bitBudget 270698.5999999968
init bitBudget 271040.39999999676
codingParams.bitReservoir 270302.39999999676
 --------- 
.init bitBudget 271584.19999999675
init bitBudget 271890.99999999674
codingParams.bitReservoir 270952.99999999674
 --------- 
.init bitBudget 272234.7999999967
init bitBudget 272597.5999999967
codingParams.bitReservoir 271640.5999999967
 --------- 
.init bitBudget 272922.3999999967
init bitBudget 273245.1999999967
codingParams.bitReservoir 272454.1999999967
 --------- 
.init bitBudget 273735.9999999967
init bitBudget 274347.79999999667
codingParams.bitReservoir 273615.79999999667
 --------- 
.init bitBudget 274897.59999999666
init bitBudget 275416.399

init bitBudget 329317.7999999953
codingParams.bitReservoir 328357.7999999953
 --------- 
.init bitBudget 329639.59999999526
init bitBudget 330263.39999999525
codingParams.bitReservoir 329515.39999999525
 --------- 
.init bitBudget 330797.19999999524
init bitBudget 331289.9999999952
codingParams.bitReservoir 330511.9999999952
 --------- 
.init bitBudget 331793.7999999952
init bitBudget 332158.5999999952
codingParams.bitReservoir 331238.5999999952
 --------- 
.init bitBudget 332520.3999999952
init bitBudget 333052.1999999952
codingParams.bitReservoir 332323.1999999952
 --------- 
.init bitBudget 333604.99999999517
init bitBudget 334111.79999999516
codingParams.bitReservoir 333336.79999999516
 --------- 
.init bitBudget 334618.59999999515
init bitBudget 335213.39999999513
codingParams.bitReservoir 334477.39999999513
 --------- 
.init bitBudget 335759.1999999951
init bitBudget 336311.9999999951
codingParams.bitReservoir 335522.9999999951
 --------- 
.init bitBudget 336804.7999999951
init b

...

.init bitBudget 514852.39999999065
bits used 1013
init bitBudget 515121.19999999064
bits used 1027
codingParams.bitReservoir 514123.19999999064
 --------- 
.init bitBudget 515404.9999999906
bits used 1058
init bitBudget 515637.7999999906
bits used 1117
codingParams.bitReservoir 514532.7999999906
 --------- 
.init bitBudget 515814.5999999906
init bitBudget 516300.3999999906
codingParams.bitReservoir 515454.3999999906
 --------- 
.init bitBudget 516736.1999999906
init bitBudget 517053.99999999057
codingParams.bitReservoir 516102.99999999057
 --------- 
.init bitBudget 517384.79999999056
init bitBudget 517693.59999999055
codingParams.bitReservoir 517000.59999999055
 --------- 
.init bitBudget 518282.39999999054
init bitBudget 518832.1999999905
codingParams.bitReservoir 518066.1999999905
 --------- 
.init bitBudget 519347.9999999905
init bitBudget 519849.7999999905
codingParams.bitReservoir 519038.7999999905
 --------- 
.init bitBudget 520320.5999999905
bits used 1019
init bitBudget 520590

codingParams.bitReservoir 570681.5999999953
 --------- 
.init bitBudget 571963.3999999954
init bitBudget 572556.1999999954
codingParams.bitReservoir 571759.1999999954
 --------- 
.init bitBudget 573040.9999999955
init bitBudget 573511.7999999955
codingParams.bitReservoir 572781.7999999955
 --------- 
.init bitBudget 574063.5999999956
init bitBudget 574611.3999999956
codingParams.bitReservoir 573863.3999999956
 --------- 
.init bitBudget 575145.1999999956
init bitBudget 575693.9999999957
codingParams.bitReservoir 574928.9999999957
 --------- 
.init bitBudget 576210.7999999957
init bitBudget 576707.5999999958
codingParams.bitReservoir 575921.5999999958
 --------- 
.init bitBudget 577203.3999999958
init bitBudget 577703.1999999959
codingParams.bitReservoir 577020.1999999959
 --------- 
.init bitBudget 578301.9999999959
init bitBudget 578854.799999996
codingParams.bitReservoir 578128.799999996
 --------- 
.init bitBudget 579410.599999996
init bitBudget 579957.3999999961
codingParams.bitRes

.codingParams.bitReservoir None
 --------- 
.codingParams.bitReservoir None
 --------- 
.codingParams.bitReservoir None
 --------- 
.codingParams.bitReservoir None
 --------- 
.codingParams.bitReservoir None
 --------- 
.codingParams.bitReservoir None
 --------- 
.codingParams.bitReservoir None
 --------- 
.codingParams.bitReservoir None
 --------- 
.codingParams.bitReservoir None
 --------- 
.codingParams.bitReservoir None
 --------- 
.codingParams.bitReservoir None
 --------- 
.codingParams.bitReservoir None
 --------- 
.codingParams.bitReservoir None
 --------- 
.codingParams.bitReservoir None
 --------- 
.codingParams.bitReservoir None
 --------- 
.codingParams.bitReservoir None
 --------- 
.codingParams.bitReservoir None
 --------- 
.codingParams.bitReservoir None
 --------- 
.codingParams.bitReservoir None
 --------- 
.codingParams.bitReservoir None
 --------- 
.codingParams.bitReservoir None
 --------- 
.codingParams.bitReservoir None
 --------- 
.codingParams.bitReservoir None


.codingParams.bitReservoir None
 --------- 
.codingParams.bitReservoir None
 --------- 
.codingParams.bitReservoir None
 --------- 
.codingParams.bitReservoir None
 --------- 
.codingParams.bitReservoir None
 --------- 
.codingParams.bitReservoir None
 --------- 
.codingParams.bitReservoir None
 --------- 
.codingParams.bitReservoir None
 --------- 
.codingParams.bitReservoir None
 --------- 
.codingParams.bitReservoir None
 --------- 
.codingParams.bitReservoir None
 --------- 
.codingParams.bitReservoir None
 --------- 
.codingParams.bitReservoir None
 --------- 
.codingParams.bitReservoir None
 --------- 
.codingParams.bitReservoir None
 --------- 
.codingParams.bitReservoir None
 --------- 
.codingParams.bitReservoir None
 --------- 
.codingParams.bitReservoir None
 --------- 
.codingParams.bitReservoir None
 --------- 
.codingParams.bitReservoir None
 --------- 
.codingParams.bitReservoir None
 --------- 
.codingParams.bitReservoir None
 --------- 
.codingParams.bitReservoir None


.codingParams.bitReservoir None
 --------- 
.codingParams.bitReservoir None
 --------- 
.codingParams.bitReservoir None
 --------- 
.codingParams.bitReservoir None
 --------- 
.codingParams.bitReservoir None
 --------- 
.codingParams.bitReservoir None
 --------- 
.codingParams.bitReservoir None
 --------- 
.codingParams.bitReservoir None
 --------- 
.codingParams.bitReservoir None
 --------- 
.codingParams.bitReservoir None
 --------- 
.codingParams.bitReservoir None
 --------- 
.codingParams.bitReservoir None
 --------- 
.codingParams.bitReservoir None
 --------- 
.codingParams.bitReservoir None
 --------- 
.codingParams.bitReservoir None
 --------- 
.codingParams.bitReservoir None
 --------- 
.codingParams.bitReservoir None
 --------- 
.codingParams.bitReservoir None
 --------- 
.codingParams.bitReservoir None
 --------- 
.codingParams.bitReservoir None
 --------- 
.codingParams.bitReservoir None
 --------- 
.codingParams.bitReservoir None
 --------- 
.codingParams.bitReservoir None
